In [173]:
import pandas as pd 
import numpy as np
import warnings
import matplotlib.pyplot as plt

In [174]:
#warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

df =  pd.read_excel("C:\\Users\\ziad.saad\\Desktop\\awp leads.xlsx")

C:\Users\ziad.saad\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [175]:
# Filtering the data from 01-10-2024

df['Created On'] = pd.to_datetime(df['Created On'])
df = df[df['Created On'] >= '10/01/2024  00:00:00']

print(len(df))
#df.head(1)

6960


In [176]:
def remove_cols(df,cols):
    df.drop(cols, axis=1, inplace=True)
    return df

cols = ["(Do Not Modify) Lead","(Do Not Modify) Row Checksum", '(Do Not Modify) Modified On','First Name','Middle Name','Last Name','Email',
            'Gender','Job Title','DoctorBugdet','Pain','Gain','Description', 'Last Appointment Description','Last Phone Call Description',
            'Name (Owning Business Unit) (Business Unit)','Country','Status Reason','Created By']

remove_cols(df,cols)

,Name,Topic,Mobile Phone,Speciality,Lead Source,Name (Source Campaign) (Campaign),Count Appointments,Appointment Result,Count PhoneCalls,Reached Phone Count,Not Reached Phone Count,Last Reached Call Result,Last Call Status,Owner,Created On,Last Reached Call Date,Re-Assigned Date,Follow Up Date
0,Walaa,Andalusia Clinic,01006256954,Nutrition,WhatsApp,ARC,0,NaN,0,NaN,NaN,NaN,NaN,Youssef Nasser,2025-02-12 15:44:27,NaT,NaT,NaT
1,Mohamed Khaled,ARC,01060734625,OB & GYN,Inbound,ARC,0,NaN,1,1.0,NaN,Unsure,Reached,Youssef Nasser,2025-02-12 15:08:55,2025-02-12 15:10:26,NaT,NaT
2,Reema Abdullah,AMC,00539068123,General Surgery,Facebook,AMC,0,NaN,1,NaN,1.0,NaN,Not Reached,Moustafa Ibrahim,2025-02-12 12:35:13,NaT,2025-02-12 14:49:53,NaT
3,Moustafa Mourad,Andalusia Clinic,01222191558,NaN,WhatsApp,ARC & AMC,0,NaN,1,1.0,NaN,Interested,Reached,Mostafa Hassan,2025-02-12 12:18:05,2025-02-12 15:29:45,2025-02-12 14:49:22,2025-02-13
4,Nour Mandouh,Andalusia Clinic,01156064069,Nutrition,WhatsApp,ARC & AMC,0,NaN,1,1.0,NaN,Unsure,Reached,Esraa Faisal Wafa Ahmed,2025-02-12 12:16:35,2025-02-12 15:09:52,2025-02-12 14:48:57,2025-02-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6955,Amira shawky,Andalusia Clinic,01032706540,Nutrition,Facebook,ARC & AMC,0,NaN,4,4.0,NaN,Not Interested,Reached,Esraa Faisal Wafa Ahmed,2024-10-01 08:51:11,2025-01-08 16:20:39,2024-12-31 13:09:01,NaT
6956,محمد نصار,Andalusia Clinic,01070143978,Internal Medicine,Facebook,ARC & AMC,0,NaN,6,4.0,2.0,Unsure,Reached,Esraa Faisal Wafa Ahmed,2024-10-01 08:50:15,2024-12-01 14:56:42,2024-12-31 13:09:00,NaT
6957,Mohammad Talaat,AMC,01121154812,Urology,Facebook,AMC,0,NaN,1,1.0,NaN,Not Interested,Reached,Esraa Faisal Wafa Ahmed,2024-10-01 07:06:00,2024-10-02 11:12:57,2024-12-31 13:09:00,NaT
6958,Abeer Ibrahim,AMC,01211398587,Dental,Facebook,AMC,0,NaN,2,2.0,NaN,Not Interested,Reached,Esraa Faisal Wafa Ahmed,2024-10-01 06:50:04,2024-10-03 13:46:12,2024-12-31 13:09:00,NaT


In [177]:
def check(data):
    details = []
    cols = data.columns
    for col in cols:
        dtypes = data[col].dtypes
        nulls = data[col].isnull().sum()
        uniques = data[col].nunique()
        details.append([col,dtypes, nulls, uniques])
    df = pd.DataFrame(details)
    df.columns = ["Column name","Data type", "Sum of null values","Unique values"]
    return df

In [178]:
check(df)

,Column name,Data type,Sum of null values,Unique values
0,Name,object,0,6630
1,Topic,object,3267,27
2,Mobile Phone,object,0,6947
3,Speciality,object,340,50
4,Lead Source,object,0,11
5,Name (Source Campaign) (Campaign),object,0,3
6,Count Appointments,int64,0,5
7,Appointment Result,object,6711,9
8,Count PhoneCalls,int64,0,13
9,Reached Phone Count,float64,4662,8


In [179]:
df.rename(columns={"Name (Source Campaign) (Campaign)":"Source campaign", ' Name':"Name"}, inplace=True)
#check(df)

In [180]:
def fill_nulls(df,col,value):
    df.fillna({col:value} ,inplace=True ,axis=0)

fill_nulls(df,'Speciality','Not specified')
fill_nulls(df,'Topic','Not specified')
fill_nulls(df,'Reached Phone Count',0)
fill_nulls(df,'Not Reached Phone Count',0)

In [181]:
# replace the null values in appointment result to (No appointment)
index = 0

for i in df["Count Appointments"]:
    if i == 0 and (pd.isna(df['Appointment Result'][index])):
        df.loc[index,"Appointment Result"] = "No appointment"
    elif i != 0 and (pd.isna(df['Appointment Result'][index])):
        df.loc[index,"Appointment Result"] = "Not mentioned"     

    index +=1

In [182]:
classify = df['Lead Source'].apply(lambda x : 'Digital' if x in ['Facebook','WhatsApp','Linkedin','Instgram','Google Ads','Tiktok'] else 'Direct')
    
df['Lead Classification'] = classify    
df['Lead Classification'].value_counts()

Lead Classification
Direct     4550
Digital    2410
Name: count, dtype: int64

In [183]:
cols = ['Name', 'Topic', 'Mobile Phone','Last 3 digits', 'Speciality', 'Lead Source', 'Lead Classification','Source campaign', 'Count Appointments',
       'Appointment Result', 'Count PhoneCalls', 'Reached Phone Count','Not Reached Phone Count','Last Reached Call Result',
        'Last Call Status', 'Owner', 'Created On','Last Reached Call Date', 'Re-Assigned Date','Follow Up Date']

df['Last 3 digits'] = df['Mobile Phone'].str[-3:]

df = df[cols]

In [184]:
# Checks the wrong entry of data, by making sure that the reached and unreached calls don't exceed Count calls

counter = 0
for x,y,z in zip(df["Count PhoneCalls"], df["Reached Phone Count"],df["Not Reached Phone Count"]):
    if (y+z > x) or (y+z < x):
        print(f"error in row number : {counter} \n       {z}  +  {y}  =  {x} ")
        print(f'{df['Name'][counter]} \n -----------------------------')
    counter +=1

error in row number : 802 
       0.0  +  5.0  =  10 
Ahmed el bahery 
 -----------------------------
error in row number : 977 
       3.0  +  5.0  =  9 
ahmed sabbagh 
 -----------------------------
error in row number : 984 
       2.0  +  1.0  =  4 
MįĐø Ãlï 
 -----------------------------
error in row number : 986 
       3.0  +  0.0  =  4 
Nesma Tarek 
 -----------------------------
error in row number : 1449 
       0.0  +  0.0  =  1 
Nada Magdy 
 -----------------------------
error in row number : 4394 
       2.0  +  3.0  =  0 
Ahmed Albehairy 
 -----------------------------
error in row number : 6233 
       1.0  +  2.0  =  4 
Mohammed Ahmed Tawfik 
 -----------------------------


In [185]:
####################################
'''
row, counter = 2, 0

for name, number in zip(df['Name'], df['Mobile Phone']):
    number = number[-5:]  
    
    phone['Last 5 Digits'] = phone['Mobile Last 5 digits (Regarding) (Lead)'].astype(str).str[-5:]

    if (name in phone['Name'].values) and (number in phone['Last 5 Digits'].values):
        matching_rows = phone[(phone['Name'] == name) & (phone['Last 5 Digits'] == number)]
        length = len(matching_rows)
        
        print(f"Match found for {name}: {number} :{length}")
    
        if length != df['Count PhoneCalls'][counter]:
            print(name)
            print(f'Error in lead number: {row} \nfound = {df['Count PhoneCalls'][counter]} \nreal = {length}  and {counter}\n ------')
            df['Count PhoneCalls'][counter] = length

    counter += 1
    row += 1  
    '''

'\nrow, counter = 2, 0\n\nfor name, number in zip(df[\'Name\'], df[\'Mobile Phone\']):\n    number = number[-5:]  \n    \n    phone[\'Last 5 Digits\'] = phone[\'Mobile Last 5 digits (Regarding) (Lead)\'].astype(str).str[-5:]\n\n    if (name in phone[\'Name\'].values) and (number in phone[\'Last 5 Digits\'].values):\n        matching_rows = phone[(phone[\'Name\'] == name) & (phone[\'Last 5 Digits\'] == number)]\n        length = len(matching_rows)\n        \n        print(f"Match found for {name}: {number} :{length}")\n    \n        if length != df[\'Count PhoneCalls\'][counter]:\n            print(name)\n            print(f\'Error in lead number: {row} \nfound = {df[\'Count PhoneCalls\'][counter]} \nreal = {length}  and {counter}\n ------\')\n            df[\'Count PhoneCalls\'][counter] = length\n\n    counter += 1\n    row += 1  \n    '

In [186]:
#check(df)

In [187]:
#- For the leads with 0 calls, which is new to the data, convert the null values to (new lead)
#- For leads with > 0 calls, which is old leads, convert the null values to (Not reached)

counter = 0

for x,y in zip(df["Count PhoneCalls"], df["Not Reached Phone Count"]):
    if (x == 0):
        df.loc[counter,"Last Reached Call Result"] = "Untouched"
    elif (x==y) and (y+x >0):
        df.loc[counter,"Last Reached Call Result"] = "Not Reached"
    counter +=1

df['Last Reached Call Result'].fillna(df['Last Call Status'], inplace=True)

C:\Users\ziad.saad\AppData\Local\Temp\ipykernel_30872\1773239341.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Last Reached Call Result'].fillna(df['Last Call Status'], inplace=True)
C:\Users\ziad.saad\AppData\Local\Temp\ipykernel_30872\1773239341.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Last Reached Call Result'].fi

In [188]:
# convert the null status values to (In progress / new lead) if the last reached call result in in progress

counter = 0

for i in df["Last Reached Call Result"]:
    if (i == "Untouched"):
        df.loc[counter,"Last Call Status"] = i
    elif i == "Unsure":
        df.loc[counter,'Last Call Status'] = "Reached"
    counter +=1

In [189]:
'''
counter = 0
for i in df["Mobile Phone"]:
    if len(str(i)) == 10 or len(str(i)) == 8:
        df["Country"][counter] = "Egypt"
    counter+=1 '''

'\ncounter = 0\nfor i in df["Mobile Phone"]:\n    if len(str(i)) == 10 or len(str(i)) == 8:\n        df["Country"][counter] = "Egypt"\n    counter+=1 '

In [190]:
# check duplicates in data 
df[df[['Name','Mobile Phone','Speciality']].duplicated()]

,Name,Topic,Mobile Phone,Last 3 digits,Speciality,Lead Source,Lead Classification,Source campaign,Count Appointments,Appointment Result,Count PhoneCalls,Reached Phone Count,Not Reached Phone Count,Last Reached Call Result,Last Call Status,Owner,Created On,Last Reached Call Date,Re-Assigned Date,Follow Up Date
950,محمد زكريا,AMC,01097595453,453,Dental,Facebook,Digital,AMC,0,No appointment,1,1.0,0.0,Not Interested,Reached,Esraa Faisal Wafa Ahmed,2025-01-05 09:40:33,2025-01-08 12:27:21,2025-01-05 15:11:37,NaT
6124,صفاء الطوخى,Andalusia Clinic,00966561756365,365,Dental,Facebook,Digital,AMC,0,No appointment,6,1.0,5.0,Not Interested,Not Reached,Youssef Nasser,2024-11-04 15:45:25,2024-11-11 14:43:55,2024-11-04 16:05:48,2025-02-11
6127,Rawia Abd El Rooaf,Andalusia Clinic,00966537839649,649,Not specified,Facebook,Digital,ARC & AMC,0,No appointment,6,1.0,5.0,Not Interested,Not Reached,Youssef Nasser,2024-11-04 15:41:50,2024-11-11 14:45:56,2024-11-04 16:05:17,2025-02-11


In [191]:
# created columns touchability

touch = []
for i,j in zip(df['Count PhoneCalls'],df['Count Appointments']):
    if (i >0) or (j >0):
        touch.append('Touched')
    else:
        touch.append('Untouched')

df['Touchability'] = touch
df['Touchability'].value_counts()

C:\Users\ziad.saad\AppData\Local\Temp\ipykernel_30872\2339353072.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Touchability'] = touch


Touchability
Untouched    3499
Touched      3461
Name: count, dtype: int64

In [192]:
# created columns Reachability

reach = []
for i in df['Reached Phone Count']:
    if (i >0):
        reach.append('Reachad')
    else:
        reach.append('Not Reached')

df['Reachability'] = reach
df['Reachability'].value_counts()

C:\Users\ziad.saad\AppData\Local\Temp\ipykernel_30872\3349252741.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Reachability'] = reach


Reachability
Not Reached    4662
Reachad        2298
Name: count, dtype: int64

In [193]:
# Rearrange the columns again
cols = ['Name', 'Topic', 'Mobile Phone', 'Last 3 digits', 'Speciality', 'Lead Source', 'Lead Classification','Source campaign',
        'Count Appointments', 'Appointment Result', 'Count PhoneCalls','Reached Phone Count', 'Not Reached Phone Count',
        'Last Reached Call Result','Last Call Status','Touchability','Reachability', 'Owner', 'Created On','Last Reached Call Date',
        'Re-Assigned Date','Follow Up Date']

df = df[cols]

In [194]:
check(df)

,Column name,Data type,Sum of null values,Unique values
0,Name,object,0,6630
1,Topic,object,0,28
2,Mobile Phone,object,0,6947
3,Last 3 digits,object,0,1003
4,Speciality,object,0,51
5,Lead Source,object,0,11
6,Lead Classification,object,0,2
7,Source campaign,object,0,3
8,Count Appointments,int64,0,5
9,Appointment Result,object,0,11


In [195]:
df.to_csv("new awp leads.csv", encoding="utf-8", index=False)

In [201]:
groups = df.groupby(['Lead Source', 'Name']).size().reset_index(name='Count')
groups.head(19)

,Lead Source,Name,Count
0,Direct,Abd El Aaty Gadallah,1
1,Direct,Abd El Alim Abou Bakr El Badrawy,1
2,Direct,Abd El Fattah Abd El Fattah Mohamed El Fetyany,1
3,Direct,Abd El Fattah Mohamed Abo El Fetouh El Tayr,1
4,Direct,Abd El Ghaffar Aabed El Geziry,1
5,Direct,Abd El Kareem Elias El Geram,1
6,Direct,Abd El Latif Mohamed Abd El Latif,1
7,Direct,Abd El Moneim Abd El Aleem El Mogy,1
8,Direct,Abd El Rahman Badwy Hasnany,1
9,Direct,Abd Rabo Hassan Abd Rabo,1
